In [0]:
from pyspark.sql import functions as F

bronze_path = 'bikestore.bronze'
silver_path = 'bikestore.silver'

# Calcula o estoque total por produto
stock = (
    spark.table(bronze_path+".stocks")
    .groupBy("product_id")
    .agg(F.sum("quantity").alias("total_stock"))
)

# Carrega as tabelas bronze
products = spark.table(bronze_path+".products")
categories = spark.table(bronze_path+".categories")
brands = spark.table(bronze_path+".brands")

# Faz os joins
df_product_silver = (
    products
    .join(categories, products.category_id == categories.category_id, "left")
    .join(brands, products.brand_id == brands.brand_id, "left")
    .join(stock, products.product_id == stock.product_id, "left")
    .select(
        products.product_id,
        products.product_name,
        brands.brand_name,
        categories.category_name,
        products.model_year,
        products.list_price,
        stock.total_stock
    )
)

df_product_silver.write.mode("overwrite").\
                    option('mergeSchema', 'true').\
                    saveAsTable(silver_path+".products")